## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path

In [2]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [3]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_devars(query)
                
                result_status = retrieve_devars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(600)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json
EGFR_advanced
https://arax.ncats.io/?source=ARS&id=9a7bba94-d019-473a-9f95-0c19eef97b4f
Running
Error
ara-aragorn Error 0
Running
ara-arax Running 0
Running
ara-bte Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=dee75850-8064-44b7-93d8-15ce3d0c9bd9
Running
Running
ara-aragorn Running 0
Running
ara-arax Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json
A.1_RHOBTB2
https://arax.ncats.io/?source=ARS&id=01443031-03d7-4e1d-889b-1b1f6f2ec2fb
Running
Running
ara-aragorn Running 0
Running
ara-arax Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi.json
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
https://arax.ncats.io/?source=ARS&id=0a4a2ff5-2bf1-4784

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_devars_results(val)
    
    workflow_result_messages[keys] = result_status

EGFR_advanced 9a7bba94-d019-473a-9f95-0c19eef97b4f
Done
Error
ara-aragorn Error 0
Error
ara-arax Error 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
kp-cohd Error 0
Done
kp-chp Done 0
Done
kp-icees Done 0
Done
kp-icees-dili Done 0
A.8_EGFR_simple dee75850-8064-44b7-93d8-15ce3d0c9bd9
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 453
Done
ara-bte Done 0
Done
ara-unsecret Done 63
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
kp-openpredict Error 0
Error
kp-cohd Error 0
Done
kp-chp Done 0
Done
kp-icees Done 0
Done
kp-icees-dili Done 0
A.1_RHOBTB2 01443031-03d7-4e1d-889b-1b1f6f2ec2fb
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 9
Done
ara-bte Done 8
Done
ara-unsecret Done 0
Done
kp-genet

In [8]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
            final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [9]:
df

,A.1_RHOBTB2,A.8_EGFR_simple,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin,EGFR_advanced
ara-aragorn,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
ara-arax,Results,Results,Results,Error,No Results,No Results,No Results,Results,Results,Results,Error,Results,Results,Error,Error,Error,Error,Error
ara-bte,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Results,Error,Error,Error,Error,No Results
ara-unsecret,No Results,Results,No Results,No Results,No Results,Error,No Results,Results,No Results,Error,Error,Error,Error,Results,No Results,Results,Results,No Results
kp-genetics,No Results,No Results,Error,Error,Error,Error,Error,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error,No Results
kp-molecular,No Results,No Results,Error,Error,Error,Error,Error,Results,No Results,Results,Error,Error,Error,No Results,Error,Error,Error,No Results
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Results,Error,Results,Results,Results,Results,Results,Results,ARS Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Results,No Results,No Results,Error,Error,No Results,Error,Error,No Results,No Results
kp-textmining,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,Error,Error,Error,Error,No Results


In [11]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [12]:
df.to_csv('ara_worklow_progress_tracker_Aug5.xlsx')

In [13]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [14]:
### save all the ids for ara submission
with open('ara_worklow_IDs_Aug6.txt', 'w') as f:
    print(dict_workflows, file=f)

In [25]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [30]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [33]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
wks_name = 'Workflow Progress Tracker'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker' id:500924345>